# Step 1. Prediction target protein structure with ESMFold


The example uses malic enzyme from Trypanosoma cruzi, which is the cause of Chagas disease. Ligand (PDB ID SZD) is one of the inhibitors in Scheme 1 in [this paper](https://pubs.acs.org/doi/epdf/10.1021/acsinfecdis.1c00231)

Even though we do have the crystal structure of the protein-ligand complex (PDB ID 6W59), we will still use ESMFold to predict the protein structure for demo purpose.

The sequence of the protein is as follows:

```
>6W59_SZD_protein_A
AILTDRYINRGTAFTMEERQKLHILGRLPPVVETLEEQVARVYGQVKKYEKPINRYQHLVSVHSTNTTLY
YATILAHLEEMLPIIYTPTVGEACMEYSHLFFRERGVYFNRLYKGQFRNIMRDAGYQKVEVVVITDGSRI
LGLGDLGSNGIGISIGKCSLYVAGAGIDPRLIVPVILDVGTNNERYLQDKDYLGMREKRLGDEEFYELLD
EFMEAASAEWPNAVIQFEDFSNNHCFDIMERYQKKYRCFNDDIQGTGAVIAAGFLNAIKLSGVSPLQQRI
VVFGAGSAAVGVANNIAALAARMYKFPVQDLVKTFYLVDTKGLVTTTRGDQLAAHKKLLARTDVSAEDSA
KLRTLEEIVRFVKPTTLLGLGGVGPAFTEEIVKMVMQNTERPIIFPLSNPTSKAEVTPENAYKWTNGAAI
VASGSPFPPTTIGGKTFKPSQGNNLYVFPGVGLGCALAQPTHIPEELLLTASESLNLLTTEGDLREGRLY
PPLEDIHNISANVATDVILEAQRMKIDNNKKLPRTRDELLAFVKKAMWKPVYSG
```


In [ ]:
# !pip install bionemo bionemo-controlled-generation biopython python-dotenv pandas numpy matplotlib loguru py3dmol
!pip install pandas numpy matplotlib loguru py3dmol

In [ ]:
import os, shutil
from google.colab import userdata
import requests
import py3Dmol
import os

In [ ]:
def prepare_output_directory(output):
    """
    Prepare the output directory
    output: str, the output directory
    return: None
    """
    # overwrite the output directory
    # delete the output directory if it exists
    if os.path.exists(output):
        shutil.rmtree(output)
    os.makedirs(output)

## 1.2 Setup output directory

In [ ]:
# overwrite the output directory
output_dir = "/content/output/esmfold_result"
prepare_output_directory(output_dir)

In [ ]:
invoke_url = "https://health.api.nvidia.com/v1/biology/nvidia/esmfold"

In [ ]:
API_KEY = userdata.get('API_KEY')
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Accept": "application/json",
}

print(headers)

## 1.4 Predict

In [ ]:
# sequence of the protein
sequence = "AILTDRYINRGTAFTMEERQKLHILGRLPPVVETLEEQVARVYGQVKKYEKPINRYQHLVSVHSTNTTLYYATILAHLEEMLPIIYTPTVGEACMEYSHLFFRERGVYFNRLYKGQFRNIMRDAGYQKVEVVVITDGSRILGLGDLGSNGIGISIGKCSLYVAGAGIDPRLIVPVILDVGTNNERYLQDKDYLGMREKRLGDEEFYELLDEFMEAASAEWPNAVIQFEDFSNNHCFDIMERYQKKYRCFNDDIQGTGAVIAAGFLNAIKLSGVSPLQQRIVVFGAGSAAVGVANNIAALAARMYKFPVQDLVKTFYLVDTKGLVTTTRGDQLAAHKKLLARTDVSAEDSAKLRTLEEIVRFVKPTTLLGLGGVGPAFTEEIVKMVMQNTERPIIFPLSNPTSKAEVTPENAYKWTNGAAIVASGSPFPPTTIGGKTFKPSQGNNLYVFPGVGLGCALAQPTHIPEELLLTASESLNLLTTEGDLREGRLYPPLEDIHNISANVATDVILEAQRMKIDNNKKLPRTRDELLAFVKKAMWKPVYSG"

print(f"length of protein: {len(sequence)} aa")

In [ ]:
payload = {
  "sequence": sequence
}

In [ ]:
# re-use connections
session = requests.Session()
response = session.post(invoke_url, headers=headers, json=payload)
response.raise_for_status()
result = response.json()

## 1.5 Analysis & visualization

In [ ]:
result.keys()

In [ ]:
# only 1 sequence is allowed each time
assert len(result["pdbs"]) == 1

In [ ]:
# Write PDB file
fp = os.path.join(output_dir, f"predicted_protein.pdb")
with open(fp, "w") as f:
    f.write(result["pdbs"][0]) # the str representation of the PDB file

We can take a look at the predicted protein structure and its pLDDT score. The score is stored as B-factor in the returned results.

To align with the [EBI/AlphaFold database](https://alphafold.ebi.ac.uk/), we will use the following colors for the pLDDT score

![plddt](https://res.cloudinary.com/dpfqlyh21/image/upload/v1705026011/obsidian/izrfmiepbzpnzm2aoqwh.png)

We display the predicted structure with these colors using py3Dmol.


In [ ]:
def load_protein(pdb_file_path, width=800, height=600):

    """
    Load a protein structure from a PDB file and display it using py3Dmol.
    pdb_file_path: str, path to the PDB file
    width: int, width of the viewer in pixels
    height: int, height of the viewer in pixels
    return: py3Dmol.view object
    """

    with open(pdb_file_path) as ifile:
        pdb_data = "".join([x for x in ifile])

    view = py3Dmol.view(width=width, height=height)
    view.addModelsAsFrames(pdb_data)

    for line in pdb_data.split("\n"):
        split = line.split()
        if len(split) == 0 or split[0] != "ATOM":
            continue
        # Assuming the B-factor is at position 10 (you may need to adjust this based on your PDB format)
        b_factor = float(split[10])
        if b_factor > 90:
            color = "blue"
        elif 70 <= b_factor <= 90:
            color = "cyan"
        elif 50 <= b_factor < 70:
            color = "yellow"
        else:
            color = "orange"

        # Atom serial numbers typically start from 1, hence idx should be used directly
        idx = int(split[1])

        # Style should be set per atom id
        view.setStyle({'model': -1, 'serial': idx}, {"cartoon": {'color': color}})
    view.zoomTo()
    return view

In [ ]:
view = load_protein(
    pdb_file_path = '/content/output/esmfold_result/predicted_protein.pdb',
    width=800,
    height=500
)
view.show()

# **Use py3Dmol to visualize the superimposed structure**
To show how the predicted structure aligns with the crystal structure, we will use the crystal structure from (PDB ID 6W59) and calcualte the backbone RMSD.

In the next two cells below, install PyMOL in Colab and then test if installed successfully by following tutorial here # https://github.com/MooersLab/colabOpenSourcePyMOLpySnips?tab=readme-ov-file. PyMOL will be used to align the predicted and the ground truth pdb files.

In [ ]:
from IPython.utils import io
import tqdm.notebook
import os
"""The PyMOL installation is done inside two nested context managers. This approach
was inspired by Dr. Christopher Schlick's (of the Phenix group at
Lawrence Berkeley National Laboratory) method for installing cctbx
in a Colab Notebook. He presented his work on September 1, 2021 at the IUCr
Crystallographic Computing School. I adapted Chris's approach here. It replaces my first approach
that requires seven steps. My approach was presentated at the SciPy2021 conference
in July 2021 and published in the
[proceedings](http://conference.scipy.org/proceedings/scipy2021/blaine_mooers.html).
The new approach is easier for beginners to use. The old approach is easier to debug
and could be used as a back-up approach.

Thank you to Professor David Oppenheimer of the University of Florida for suggesting the use mamba and of Open Source PyMOL.
"""
total = 100
with tqdm.notebook.tqdm(total=total) as pbar:
    with io.capture_output() as captured:

        !pip install -q condacolab
        import condacolab
        condacolab.install()
        pbar.update(10)

        import sys
        sys.path.append('/usr/local/lib/python3.10/site-packages/')
        pbar.update(20)

        # Install PyMOL
        %shell mamba install pymol-open-source --yes

        pbar.update(100)

In [ ]:
# test if installation is successful
# If the test failed, try to run the installation cell above again
from pymol import cmd
from IPython.display import Image
cmd.fetch("1lw9")
cmd.orient("1lw9")
cmd.rotate("z","270")
cmd.png("test.png")
Image(filename = "test.png", unconfined=True)

In [ ]:
import py3Dmol
from pymol import cmd
import os

def align_protein(pred_pdb_file, true_pdb_file, output_dir, pred_color = 'cyan', true_color = 'green', width=800, height=600):
    """
    Align two protein structures and display them using py3Dmol.
    pred_pdb_file: str, path to the PDB file of the predicted protein
    true_pdb_file: str, path to the PDB file of the true protein
    output_dir: str, path to the output directory to save the aligned PDB files
    pred_color: str, color of the predicted protein in the viewer
    true_color: str, color of the true protein in the viewer
    width: int, width of the viewer in pixels
    height: int, height of the viewer in pixels
    return: py3Dmol.view object
    """

    # always reinitialize!
    cmd.reinitialize()

    cmd.load(pred_pdb_file, 'predicted_protein')
    cmd.load(true_pdb_file, 'true_protein')

    # Align the structures on the backbone atoms
    cmd.align('predicted_protein & backbone', 'true_protein & backbone', quiet=0)

    # Save the aligned structures
    pred_aligned_fp = os.path.join(output_dir, 'aligned_predicted_protein.pdb')
    true_aligned_fp = os.path.join(output_dir, 'aligned_true_protein.pdb')
    cmd.save(pred_aligned_fp, 'predicted_protein')
    cmd.save(true_aligned_fp, 'true_protein')

    # Read PDB files into strings (assuming they are in the current directory)
    with open(pred_aligned_fp, 'r') as file:
        pred_pdb_data = file.read()

    with open(true_aligned_fp, 'r') as file:
        true_pdb_data = file.read()

    # Create a py3Dmol view
    view = py3Dmol.view(width=width, height=height)

    # Add the predicted protein model
    view.addModel(pred_pdb_data, 'pdb')
    # Set a specific color for the predicted protein, e.g., green
    view.setStyle({'model': 0}, {'cartoon': {'color': pred_color}})

    # Add the true protein model
    view.addModel(true_pdb_data, 'pdb')
    # Set a different specific color for the true protein, e.g., blue
    view.setStyle({'model': 1}, {'cartoon': {'color': true_color}})

    view.zoomTo()
    return view

In [ ]:
# upload the ground truth pdb "6W59_SZD_protein.pd"
from google.colab import files
uploaded = files.upload()

In [ ]:
view = align_protein(
    pred_pdb_file = '/content/output/esmfold_result/predicted_protein.pdb', # prediction
    true_pdb_file = '/content/6W59_SZD_protein.pdb', # true crystal structure
    output_dir = '/content/output/esmfold_result', # output directory to store the aligned poses so that py3dmol can load back
    pred_color = 'cyan', # color for the predicted structure
    true_color = 'green', # color for the true structure
    width=800,
    height=500
)
view.show()

To show how the predicted structure aligns with the crystal structure, we will use the crystal structure from (PDB ID 6W59) and calculate the backbone RMSD.

You can see that the backbone RMSD is roughly about 1.4 A. According to the [ESMFold paper](https://www.science.org/doi/10.1126/science.ade2574), on the CAMEO test set, ESMFold predictions
have a median all-atom RMSD95 (root-mean-squared deviation at 95% residue coverage) of 1.91 A and backbone RMSD95 of 1.33 A. Our result is similar with the reported median number.  

Note that the ESMFold prediction does not differentiate between holo/apo structures.

# **Download the predicted protein PDB file**
It will be uploaded in Step 3 DiffDock inference

In [ ]:
# from google.colab import files
pred_pdb_file = '/content/output/esmfold_result/predicted_protein.pdb'
files.download(pred_pdb_file)